In [ ]:
from google.colab import files
upload = files.upload()

Saving kaggle-2.json to kaggle-2.json


In [ ]:
!pwd

/content


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content'

In [ ]:
!kaggle competitions download -c 2022-2-iab-challenge-1

 99% 240M/242M [00:08<00:00, 26.2MB/s]
100% 242M/242M [00:08<00:00, 28.5MB/s]


In [ ]:
!unzip -qq '/content/2022-2-iab-challenge-1.zip'

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import torch.nn as nn
import os
import cv2
import glob
from torchvision import transforms
import shutil
newpath = '/kaggle/working/images'
if not os.path.exists(newpath):
    os.makedirs(newpath)
# print(os.listdir("../input/dogs-vs-cats/Data/Train/"))

train_path = '/content/train'
test_path = '/content/test'

print("type of Images in Train dataset"+str(os.listdir(train_path)))
print("type of Images in Test dataset"+str(os.listdir(test_path)))

type of Images in Train dataset['16345.jpg', '17905.jpg', '12601.jpg', '4440.jpg', '7313.jpg', '7378.jpg', '16399.jpg', '14349.jpg', '7411.jpg', '13440.jpg', '13126.jpg', '9258.jpg', '1613.jpg', '250.jpg', '7840.jpg', '14663.jpg', '5812.jpg', '13956.jpg', '14076.jpg', '15939.jpg', '13664.jpg', '2411.jpg', '3843.jpg', '6473.jpg', '2091.jpg', '13154.jpg', '18407.jpg', '13403.jpg', '1107.jpg', '8336.jpg', '17553.jpg', '14291.jpg', '17831.jpg', '10924.jpg', '7692.jpg', '8629.jpg', '489.jpg', '4158.jpg', '380.jpg', '6269.jpg', '7019.jpg', '8274.jpg', '14964.jpg', '3197.jpg', '3202.jpg', '14314.jpg', '9312.jpg', '5648.jpg', '9868.jpg', '12398.jpg', '18101.jpg', '3992.jpg', '18889.jpg', '562.jpg', '5930.jpg', '15153.jpg', '3798.jpg', '2397.jpg', '17399.jpg', '19350.jpg', '19397.jpg', '13557.jpg', '16199.jpg', '15135.jpg', '978.jpg', '11759.jpg', '16946.jpg', '6468.jpg', '3532.jpg', '14959.jpg', '685.jpg', '2320.jpg', '7553.jpg', '12623.jpg', '16807.jpg', '14005.jpg', '3861.jpg', '7531.jpg', '

In [ ]:
import pandas as pd

label = pd.read_csv('/content/train_label.csv')

In [ ]:
label

,file_name,class_label
0,7236.jpg,5
1,4217.jpg,2
2,15493.jpg,3
3,15254.jpg,3
4,16668.jpg,2
...,...,...
14029,18989.jpg,2
14030,13339.jpg,3
14031,2546.jpg,4
14032,15897.jpg,4


In [ ]:
import os
import pandas as pd
from torchvision.io import read_image
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

class CustomImageDataset(Dataset):
    def __init__(self, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv('train_label.csv')
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = plt.imread(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [ ]:
from torch.utils.data import DataLoader
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

data = CustomImageDataset('/content/train', transform=transform)
train_set, val_set = torch.utils.data.random_split(data, [0.7, 0.3])
batch_size = 32

# Create data loaders.
train_dataloader = DataLoader(train_set, batch_size=batch_size)
test_dataloader = DataLoader(val_set, batch_size=batch_size)


In [ ]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")


class Bottleneck(nn.Module):
    expansion = 4
    def __init__(self, in_channels, out_channels, i_downsample=None, stride=1):
        super(Bottleneck, self).__init__()

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0)
        self.batch_norm1 = nn.BatchNorm2d(out_channels)

        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.batch_norm2 = nn.BatchNorm2d(out_channels)

        self.conv3 = nn.Conv2d(out_channels, out_channels*self.expansion, kernel_size=1, stride=1, padding=0)
        self.batch_norm3 = nn.BatchNorm2d(out_channels*self.expansion)

        self.i_downsample = i_downsample
        self.stride = stride
        self.relu = nn.ReLU()

    def forward(self, x):
        identity = x.clone()
        x = self.relu(self.batch_norm1(self.conv1(x)))

        x = self.relu(self.batch_norm2(self.conv2(x)))

        x = self.conv3(x)
        x = self.batch_norm3(x)

        #downsample if needed
        if self.i_downsample is not None:
            identity = self.i_downsample(identity)
        #add identity
        x+=identity
        x=self.relu(x)

        return x



class ResNet(nn.Module):
    def __init__(self, ResBlock, layer_list, num_classes, num_channels=3):
        super(ResNet, self).__init__()
        self.in_channels = 64

        self.conv1 = nn.Conv2d(num_channels, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.batch_norm1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.max_pool = nn.MaxPool2d(kernel_size = 3, stride=2, padding=1)

        self.layer1 = self._make_layer(ResBlock, layer_list[0], planes=64)
        self.layer2 = self._make_layer(ResBlock, layer_list[1], planes=128, stride=2)
        self.layer3 = self._make_layer(ResBlock, layer_list[2], planes=256, stride=2)
        self.layer4 = self._make_layer(ResBlock, layer_list[3], planes=512, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(512*ResBlock.expansion, num_classes)

    def forward(self, x):
        x = self.relu(self.batch_norm1(self.conv1(x)))
        x = self.max_pool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)

        return x

    def _make_layer(self, ResBlock, blocks, planes, stride=1):
        ii_downsample = None
        layers = []

        if stride != 1 or self.in_channels != planes*ResBlock.expansion:
            ii_downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, planes*ResBlock.expansion, kernel_size=1, stride=stride),
                nn.BatchNorm2d(planes*ResBlock.expansion)
            )

        layers.append(ResBlock(self.in_channels, planes, i_downsample=ii_downsample, stride=stride))
        self.in_channels = planes*ResBlock.expansion

        for i in range(blocks-1):
            layers.append(ResBlock(self.in_channels, planes))

        return nn.Sequential(*layers)


model = ResNet(Bottleneck, [3,3,3,3], 6, 3).to(device)

Using cpu device


In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 3
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 1.745397  [    0/ 9824]
